In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import nibabel as nib
import os
from sklearn.decomposition import PCA
import math
from skimage import io
import plotly.plotly as py
from plotly.grid_objs import Grid, Column
import time
%matplotlib notebook
from mpl_toolkits.mplot3d import Axes3D
from scipy.ndimage.interpolation import rotate as rot
import glob
from scipy import ndimage
from scipy.ndimage.interpolation import zoom
import pickle
from skimage import measure

In [2]:
def read_mutant_txt(path):
    name_list = []
    fo = open(path)
    for line in fo:
        striped_line = line.strip('\n')
        if striped_line != '':
            name_list.append(striped_line)
    return name_list

def l2norm(a,b):
    return np.sqrt(a**2+b**2)


def align_img(bv_label):
    x,y,z = np.where(bv_label==1)
    points = np.vstack((x,y,z))
    points = points.T
    #print(points.shape)
    pca = PCA(n_components = 3)
    pca.fit(points)
    pc1 = pca.components_[0,:]
    #print(pc1)
    pc2 = pca.components_[1,:]
    if pc1[0] >=0 and pc1[1] >=0:
        azimuth = -np.arctan(pc1[1]/pc1[0])*(360/(2*np.pi))
    #print('case1')
#     clockwise = 1

    elif pc1[0] <=0 and pc1[1] <=0:
        azimuth = (180 - np.arctan(pc1[1]/pc1[0])*(360/(2*np.pi)))
    #    print('case2')
    #     anticlockwise = 1

    elif pc1[0]>=0 and pc1[1]<=0:
        azimuth = np.arctan(np.abs(pc1[1])/np.abs(pc1[0]))*(360/(2*np.pi))
    #    print('case3')
    #     anticlockwise = 1
    else:
        azimuth = -(90 + np.arctan(np.abs(pc1[0])/np.abs(pc1[1]))*(360/(2*np.pi)))
    #    print('case4')
    #     clockwise = 1

    # finding the elevation angle
    if pc1[2]>=0:
        elevation = -np.arctan(pc1[2]/l2norm(pc1[0],pc1[1]))*(360/(2*np.pi))
     #   print('case1')
    else:
        elevation = np.arctan(np.abs(pc1[2])/l2norm(pc1[0],pc1[1]))*(360/(2*np.pi))
     #   print('case2')
   
    bv_label_copy = bv_label.copy()
    label_rot1 = rot(bv_label_copy,angle=azimuth,axes=(0,1))
    label_rot = rot(label_rot1,angle=elevation,axes=(0,2))
    
    xr,yr,zr = np.where(label_rot==1)
    points_rot = np.vstack((xr,yr,zr))
    points_rot = points_rot.T
    #print(points_rot.shape)
    pca_rot = PCA(n_components = 3)
    pca_rot.fit(points_rot)
    pc1r = pca_rot.components_[0,:]
    pc2r = pca_rot.components_[1,:]
    
    angle3 = np.arctan(pc2r[2]/pc2r[1])*(360/(2*np.pi))
    
    label_rot = rot(label_rot,angle=-angle3,axes=(1,2))
    
    return azimuth, elevation, angle3, label_rot

def save_img(img, label, count):
    img_nft = nib.Nifti1Image(img,np.eye(4))
    img_save_data_path = './resize_img/img{}.nii'.format(count)
    nib.save(img_nft,img_save_data_path)
    
    img_nft = nib.Nifti1Image(label,np.eye(4))
    img_save_data_path = './resize_img/label{}.nii'.format(count)
    nib.save(img_nft,img_save_data_path)
    

In [3]:
data_base_path = '/scratch/zq415/grammar_cor/mutant_detect/mutant_cla/data/logitudinal'
data_folder_list = ['2016', '2017', '2018']
# data_folder_list2 = 'fix_organized_data_nii'

all_BVs = []
for cur_floder in data_folder_list:
    cur_folder_path = os.path.join(data_base_path,cur_floder)
    all_BVs += glob.glob(cur_folder_path+'/*BV*')
print(len(all_BVs))

433


In [ ]:
count = 0
mutant_label = {}

for i,img_path in enumerate(all_BVs):    
    img_label = nib.load(img_path)
    img_label = np.uint8(img_label.get_data())
    img_label[img_label>0] = 1
    
    y_predict_component = measure.label(img_label)
    component_num = np.unique(y_predict_component)
    for current_component in range(1,len(component_num)):
        if np.sum(y_predict_component == current_component) < 300:
            img_label[y_predict_component == current_component] = 0
    
    print(np.shape(img_label))
    
    azimuth, elevation, angle3, label_rot = align_img(img_label)
    
#     img_rot1 = rot(img,angle=azimuth,axes=(0,1))
#     img_rot2 = rot(img_rot1,angle=elevation,axes=(0,2))
#     img_rot = rot(img_rot2,angle=-angle3,axes=(1,2))
    
    x_slice,y_slice,z_slice = ndimage.find_objects(label_rot)[0]
    print(count, 'bv size: ', x_slice.stop-x_slice.start, y_slice.stop-y_slice.start, z_slice.stop-z_slice.start)
    
    label_slice = label_rot[x_slice.start:x_slice.stop, y_slice.start:y_slice.stop, z_slice.start:z_slice.stop]
    
    x, y, z = np.shape(label_slice)
    label_resized = zoom(label_slice, (112.0/x, 64.0/y, 64.0/z))
    label_resized[label_resized>=0.5] = 1
    label_resized[label_resized<0.5] = 0
    
#     img_resized = zoom(img_slice, (112.0/x, 64.0/y, 64.0/z))
    
    save_img(label_resized, label_resized, count)
    count += 1
    
    bv_base_name = os.path.basename(img_path)[:-14]
   
    mutant_label[i] = (i, 0, bv_base_name, label_resized, img_path)

(201, 150, 201)
0 bv size:  87 56 41
(261, 180, 281)
1 bv size:  181 112 80
(241, 180, 241)
2 bv size:  97 58 46


/share/apps/python3/3.6.3/intel/lib/python3.6/site-packages/scipy-1.0.0rc1-py3.6-linux-x86_64.egg/scipy/ndimage/interpolation.py:616: UserWarning:

From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.



(241, 210, 281)
3 bv size:  182 94 84
(221, 180, 241)
4 bv size:  109 68 53
(241, 180, 241)
5 bv size:  108 58 55
(261, 210, 241)
6 bv size:  107 50 52


/share/apps/python3/3.6.3/intel/lib/python3.6/site-packages/scipy-1.0.0rc1-py3.6-linux-x86_64.egg/scipy/ndimage/interpolation.py:616: UserWarning:

From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.



(201, 180, 201)
7 bv size:  98 45 40
(240, 180, 241)
8 bv size:  108 56 52
(221, 180, 241)
9 bv size:  101 61 50
(261, 210, 241)
10 bv size:  103 62 43
(302, 180, 261)
11 bv size:  105 110 52
(201, 180, 241)
12 bv size:  99 65 52
(302, 180, 261)
13 bv size:  103 99 49


...

(241, 180, 241)
142 bv size:  107 59 55


/share/apps/python3/3.6.3/intel/lib/python3.6/site-packages/scipy-1.0.0rc1-py3.6-linux-x86_64.egg/scipy/ndimage/interpolation.py:616: UserWarning:

From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.



(241, 210, 281)
143 bv size:  182 97 93
(261, 210, 241)
144 bv size:  109 109 59
(181, 180, 201)
145 bv size:  81 52 48
(241, 180, 241)
146 bv size:  106 54 48
(261, 210, 241)
147 bv size:  162 117 79
(221, 180, 261)
148 bv size:  98 67 55
(261, 210, 241)
149 bv size:  177 114 82
(261, 210, 241)
150 bv size:  104 101 52
(261, 210, 241)
151 bv size:  108 104 49


/share/apps/python3/3.6.3/intel/lib/python3.6/site-packages/scipy-1.0.0rc1-py3.6-linux-x86_64.egg/scipy/ndimage/interpolation.py:616: UserWarning:

From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.



(241, 210, 281)
152 bv size:  105 56 61
(332, 180, 261)


In [ ]:
save_name = 'All_data_112_64_64_longitudinal_remove_small.pickle'
save_file = open(os.path.join(os.getcwd(),'data',save_name),'wb')
pickle.dump(mutant_label,save_file)
save_file.close()


In [ ]:
mutant_names = read_mutant_txt('mutant_imgs.txt')
mutant_num = 0 
mutant_label = {}
test_mut = []
year_name = []
for i,img_path in enumerate(all_data_list):
    if 'BV' in img_path[1]:
        bv_base_name = os.path.basename(img_path[0])[:-4]
    else:
        bv_base_name = os.path.basename(img_path[0])[:-6]
        
    if bv_base_name in mutant_names:
        mutant_label[i] = (i, 0, bv_base_name, img_path[1], img_path[0])
        if '2016' in img_path[0]:
            test_mut.append(bv_base_name)
    else:
        mutant_label[i] = (i, 1, bv_base_name, img_path[1], img_path[0])
        
    if '2016' in img_path[0]:
        year_name.append(bv_base_name)
    

In [ ]:
for i, name in enumerate(mutant_names):
    print(i, ': ',name)

mutant_group = [(9,10), (12,13,14), (16,17,18,19), (36,37), (38,39), (42,43), (44,45,46,47), (48,49,50,51), (52,53), (54,55),
(56,57,58), (59,60,61), (63,64), (66,67,68), (69,70,71), (73,74), (75,76), (77,78,79), (80,81,82,83,84,85,86,87),
(89,90), (91,92), (93,94), (95,96,97), (98,99), (100,101,102)]

group_list = []
for one_group in mutant_group:
    for ii in range(len(one_group)):
        group_list.append(one_group[ii])

In [ ]:
group_list

In [ ]:
33 + len(mutant_group)